# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [1]:
!pip install datasets==2.21.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.6.1 which is incompatible.


In [2]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

The repository for vblagoje/wikipedia_snippets_streamed contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/vblagoje/wikipedia_snippets_streamed.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [3]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [4]:
history = wiki_data.filter(lambda x: x['section_title'] == 'History')


In [5]:
first_history = next(iter(history))


In [6]:
first_history

{'wiki_id': 'Q2644349',
 'start_paragraph': 10,
 'start_character': 397,
 'end_paragraph': 10,
 'end_character': 534,
 'article_title': 'Taupo District',
 'section_title': 'History',
 'passage_text': 'was not until the 1950s that the region started to develop, with forestry and the construction of the Wairakei geothermal power station.'}

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [7]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 10
counter = 0
docs = []

# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # extract the fields we need - article, section, and passage
    doc = {
        'article': d['article_title'],
        'section': d['section_title'],
        'passage': d['passage_text']
    }
    docs.append(doc)

    # increase the counter on every iteration
    counter += 1

    # Stop after collecting total_doc_count documents
    if counter >= total_doc_count:
        break

  0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
import pandas as pd
df = pd.DataFrame(docs)


In [9]:
df

,article,section,passage
0,Taupo District,History,was not until the 1950s that the region starte...
1,The Bishop Wand Church of England School,History,The Bishop Wand Church of England School Histo...
2,Surface Hill Uniting Church,History,in perpetual reminder that work and worship go...
3,The Electras (band),History,"as its B-side. However, copies of the single, ..."
4,Swanton House,History,it. Lane provided funds for restoration by the...
5,Takashinohama Line,History,Takashinohama Line The Takashinohama Line (高師浜...
6,Tamil Methodist Church,History,Tamil Methodist Church History The church was ...
7,Star Music,History,in order to strengthen its production base and...
8,Terai,History,timber reserves.\nIndian immigration increased...
9,Te Atatū (New Zealand electorate),History,first openly gay member of Parliament.\nWith t...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [10]:
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.5 MB/s eta 0:00:00


In [11]:
from google.colab import userdata
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = userdata.get('PINECONE_API_KEY')

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [12]:
import os
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [13]:
index_name = "wiki-ironhack-qa"

In [16]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes():
    # Create the index
    pc.create_index(
        name=index_name,
        metric="cosine",
        dimension=768,  # for bert-base models
        spec=spec
    )
    # Wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# Connect to the index
index = pc.Index(index_name)

# Verify index is empty
stats = index.describe_index_stats()
print(f"Index statistics: {stats}")

Index statistics: {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [17]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load the retriever model from huggingface model hub
retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')
retriever.to(device)  # Move model to GPU if available
retriever

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [18]:
# we will use batches of 64
batch_size = 64

# Create embeddings for the passage_text and include metadata in each batch
for i in tqdm(range(0, len(docs), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(docs))

    # extract batch
    batch = docs[i:i_end]

    # generate embeddings for batch
    embeddings = retriever.encode([doc['passage'] for doc in batch]).tolist()

    # create metadata and upsert batch
    metadata = [
        {
            'article': doc['article'],
            'section': doc['section'],
            'text': doc['passage']
        } for doc in batch
    ]

    # create unique IDs
    ids = [f"doc_{i + j}" for j in range(len(batch))]

    # create upsert list
    to_upsert = list(zip(ids, embeddings, metadata))

    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

# check that we have all vectors in index
print("Final index statistics:")
print(index.describe_index_stats())

  0%|          | 0/1 [00:00<?, ?it/s]

Final index statistics:
{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [19]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [20]:
def query_pinecone(query, top_k, format_results=True):
    # generate embeddings for the query
    xq = retriever.encode(query).tolist()

    # search pinecone index for context passage with the answer
    xc = index.query(
        vector=xq,
        top_k=top_k,
        include_metadata=True
    )

    if format_results:
        formatted_results = []
        for match in xc['matches']:
            formatted_results.append({
                'score': round(match['score'], 3),
                'article': match['metadata']['article'],
                'text': match['metadata']['text']
            })
        return formatted_results

    return xc

In [21]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    # Check if context is already a list or a Pinecone response
    if isinstance(context, dict) and 'matches' in context:
        # If it's a Pinecone response
        context = [f"<P> {m['metadata']['text']}" for m in context['matches']]
    else:
        # If it's already a list
        context = [f"<P> {m['metadata']['text']}" for m in context]

    # concatenate all context passages
    context = " ".join(context)

    # concatenate the query and context passages
    query = f"Question: {query} Context: {context}"

    return query

In [26]:
def format_query(query, context):
    # Format context with <P> tags, accessing 'text' and 'article' directly
    formatted_context = [f"<P> {m['text']}" for m in context]

    # Join all context passages
    joined_context = " ".join(formatted_context)

    # Create final query string
    formatted_query = f"Question: {query} Context: {joined_context}"

    return formatted_query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [27]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

[{'score': 0.363,
  'article': 'Taupo District',
  'text': 'was not until the 1950s that the region started to develop, with forestry and the construction of the Wairakei geothermal power station.'}]

In [28]:
result

[{'score': 0.363,
  'article': 'Taupo District',
  'text': 'was not until the 1950s that the region started to develop, with forestry and the construction of the Wairakei geothermal power station.'}]

In [24]:
from pprint import pprint

In [29]:
query = format_query(query, result)
pprint(query)

('Question: when was the first electric power system built? Context: <P> was '
 'not until the 1950s that the region started to develop, with forestry and '
 'the construction of the Wairakei geothermal power station.')


The output looks great. Now let's write a function to generate answers.

In [30]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [31]:
generate_answer(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('The first electric power system was built in the United States in the early '
 '1800s. The first electric power plant was built in the United Kingdom in '
 '1834.')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [32]:
def format_query(query, context):
    # Format context with <P> tags
    formatted_context = [f"<P> {m['text']}" for m in context]

    # Join all context passages
    joined_context = " ".join(formatted_context)

    # Create final query string
    formatted_query = f"Question: {query} Context: {joined_context}"

    return formatted_query

# Now use it in sequence
query = "How was the first wireless message sent?"

# Get context from Pinecone
context = query_pinecone(query, top_k=5)

# Handle both possible return types
if isinstance(context, dict) and "matches" in context:
    formatted_query = format_query(query, context["matches"])
else:
    formatted_query = format_query(query, context)

# Generate answer
answer = generate_answer(formatted_query)

('The first wireless message was sent in the early 1900s. It was sent by a '
 'telegraph, which was a small, hand-held device that could be used to send a '
 'message.')


In [33]:
print("Context structure:", context[0] if isinstance(context, list) else context["matches"][0] if isinstance(context, dict) else "Unknown structure")

Context structure: {'score': 0.091, 'article': 'Te Atatū (New Zealand electorate)', 'text': "first openly gay member of Parliament.\nWith the introduction of MMP voting in 1996, Te Atatū was scrapped in favour of a new seat called Waipareira, which covered the same area as Te Atatū, but also included the wealthy harbourside suburbs to the north of the seat. Neeson and Carter were rematched, when the presence of former Labour MP for West Auckland turned New Zealand First candidate Jack Elder, undermined Carter's chances and handed the seat to Neeson by fewer than a thousand votes.\nThree years later, Te Atatu was re-established, with the new seat focused more on the working class suburbs"}


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [34]:
for doc in context:
    print(doc["text"], end='\n---\n')

first openly gay member of Parliament.
With the introduction of MMP voting in 1996, Te Atatū was scrapped in favour of a new seat called Waipareira, which covered the same area as Te Atatū, but also included the wealthy harbourside suburbs to the north of the seat. Neeson and Carter were rematched, when the presence of former Labour MP for West Auckland turned New Zealand First candidate Jack Elder, undermined Carter's chances and handed the seat to Neeson by fewer than a thousand votes.
Three years later, Te Atatu was re-established, with the new seat focused more on the working class suburbs
---
was not until the 1950s that the region started to develop, with forestry and the construction of the Wairakei geothermal power station.
---
Tamil Methodist Church History The church was built in 1908 and initially known as Kubu Methodist Church. As the congregation grew bigger, space became an issue. In 1954, the Peranakans and English-speaking expatriates built a new church at Tengkera Stre

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [35]:
for doc in context:
    print(doc["text"], end='\n---\n')

first openly gay member of Parliament.
With the introduction of MMP voting in 1996, Te Atatū was scrapped in favour of a new seat called Waipareira, which covered the same area as Te Atatū, but also included the wealthy harbourside suburbs to the north of the seat. Neeson and Carter were rematched, when the presence of former Labour MP for West Auckland turned New Zealand First candidate Jack Elder, undermined Carter's chances and handed the seat to Neeson by fewer than a thousand votes.
Three years later, Te Atatu was re-established, with the new seat focused more on the working class suburbs
---
was not until the 1950s that the region started to develop, with forestry and the construction of the Wairakei geothermal power station.
---
Tamil Methodist Church History The church was built in 1908 and initially known as Kubu Methodist Church. As the congregation grew bigger, space became an issue. In 1954, the Peranakans and English-speaking expatriates built a new church at Tengkera Stre

Let’s finish with a final few questions.

In [36]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context)
generate_answer(query)

("I'm not sure if this counts as a project, but I'm pretty sure that the "
 'Apollo missions were the most expensive. The Apollo missions cost about $2.5 '
 'billion, and the')


As we can see, the model can generate some decent answers.